In [1]:
!pip install unsloth evaluate bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 143.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from unsloth import FastLanguageModel
import torch

# REPLACE THIS with your actual Hugging Face model ID
model_id = "michaelHenry1/Llama-3.2-3B-Instruct-bnb-4bit_finetuned"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True, # Use 4-bit to save memory during evaluation
)
FastLanguageModel.for_inference(model) # Enable faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Ll

In [4]:
# 3. Load Your Test Data
# Format: JSONL with 'conversations' or 'messages'
import json

test_data = []
with open("sample_data/holdout_evaluation_set.jsonl", "r") as f:
    for line in f:
        test_data.append(json.loads(line))

print(f"Loaded {len(test_data)} test examples.")

Loaded 1492 test examples.


### Evaluating only the last assistant response (can mistakenly assess the user response instead of the assistant response if it is the last message in the conversation)

In [ ]:
# 4. Generate Responses (The Inference Loop)
from tqdm import tqdm

references = []
predictions = []

# Using the Llama 3 format for prompting
def format_prompt(messages):
    # We only want the user's input, not the assistant's answer (that's cheating!)
    user_msg = [msg for msg in messages if msg['role'] == 'user'][0]
    return tokenizer.apply_chat_template([user_msg], tokenize=False, add_generation_prompt=True)

print("Generating responses...")
for entry in tqdm(test_data):
    # Get the "Ground Truth" (what the human actually said)
    # Assuming the last message is the assistant's target response
    ground_truth = entry['messages'][-1]['content']
    references.append(ground_truth)

    # Prepare input for the model
    prompt = format_prompt(entry['messages'])
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate answer
    outputs = model.generate(
        **inputs,
        max_new_tokens = 128,
        use_cache = True,
        pad_token_id = tokenizer.eos_token_id
    )

    # Decode the result
    # We slice [inputs.shape[1]:] to remove the prompt from the output
    generated_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
    predictions.append(generated_text)


#### Evaluating all the assistant's responses in each conversation, and keeping all previous messages of the current conversation in context.
#### using "Sliding Window" technique


In [7]:
# 4. The "Sliding Window" Data Exploder
from tqdm import tqdm
import json
import numpy as np
import evaluate
references = []
predictions = []

print("Running Sliding Window Generation...")

for entry in tqdm(test_data):
    messages = entry['messages']

    # Loop through the specific conversation to find EVERY assistant turn
    for i, msg in enumerate(messages):

        # We only care when the Assistant speaks
        if msg['role'] == "assistant":
            ground_truth = msg['content']

            # --- FILTERING OPTION ---
            # If you want to ignore short "dummy" messages like "Ok", "Thanks", etc.
            # you can uncomment the lines below:
            # if len(ground_truth.split()) < 3:
            #     continue

            # The Prompt is everything that happened BEFORE this specific turn
            # e.g., if i is 3, we take 0, 1, and 2.
            prompt_msgs = messages[:i]

            # Skip if there is no history (rare edge case)
            if len(prompt_msgs) == 0:
                continue

            references.append(ground_truth)

            # Format the prompt just for this specific moment in time
            prompt = tokenizer.apply_chat_template(
                prompt_msgs,
                tokenize=False,
                add_generation_prompt=True
            )

            inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

            # Generate response
            outputs = model.generate(
                **inputs,
                max_new_tokens = 128,
                use_cache = True,
                pad_token_id = tokenizer.eos_token_id
            )

            # Decode
            generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
            # generated_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
            predictions.append(generated_text)

print(f"\nExploded into {len(predictions)} individual evaluation turns.")

Running Sliding Window Generation...


100%|██████████| 1492/1492 [43:36<00:00,  1.75s/it]


Exploded into 4342 individual evaluation turns.


In [9]:
import json

# 1. Combine the lists into a structured format
results_data = []
for ref, pred in zip(references, predictions):
    results_data.append({
        "ground_truth": ref,
        "model_prediction": pred,
        "match": ref == pred # Quick boolean check (mostly false for generative AI)
    })

# 2. Save to a file
output_file = "evaluation_results.json"
with open(output_file, "w") as f:
    json.dump(results_data, f, indent=4)

print(f"Saved {len(results_data)} results to {output_file}")

Saved 4342 results to evaluation_results.json


In [8]:
# 5. Calculate BERTScore
import evaluate

bertscore = evaluate.load("bertscore")

print("Calculating BERTScore (this downloads a helper model)...")
results = bertscore.compute(predictions=predictions, references=references, lang="en")


Calculating BERTScore (this downloads a helper model)...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 6. Display Results
import numpy as np

print(f"Precision: {np.mean(results['precision']):.4f}")
print(f"Recall:    {np.mean(results['recall']):.4f}")
print(f"F1 Score:  {np.mean(results['f1']):.4f}")


Precision: 0.8849
Recall:    0.8693
F1 Score:  0.8769


In [15]:
# Create a list that bundles the score with the text
# Zip combines them safely since they are all the same length (the "exploded" length)
bad_examples = []
for i, f1 in enumerate(results['f1']):
    if f1 < 0.85: # Threshold for "bad"
        bad_examples.append({
            "score": f1,
            "prediction": predictions[i],
            "reference": references[i]
        })

# Sort them so the WORST ones are at the top
bad_examples.sort(key=lambda x: x["score"])

# Print the top 5 worst failures
print(f"Found {len(bad_examples)} bad examples. Here are the worst 5:")

for ex in bad_examples[:5]:
    print(f"\n--- Low Score Example (F1: {ex['score']:.4f}) ---")
    # We don't have the original User Question easily available here due to the split,
    # but usually seeing the 'Reference' is enough to know what the question was.
    print(f"Model Said: {ex['prediction']}")
    print(f"Should be:  {ex['reference']}")

Found 605 bad examples. Here are the worst 5:

--- Low Score Example (F1: 0.7254) ---
Model Said: It's 0 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8
Should be:  It's 1200. Room service is available 24 hours a day.

--- Low Score Example (F1: 0.7666) ---
Model Said: I can't find any information on a person named keith whitley.
Should be:  Jackie Keith Whitley (July 1, 1954Stambler, Irwin, and Grelun Landon (2000). - Country Music: The Encyclopedia. - New York: St. Martin's Press. - p.533. - ISBN 978-0-312-26487-1.—Carlin, Richard (2003). - Country Music: A Biographical Dictionary. - New York: Routledge - p.427. - ISBN 978-0-415-93802-0.—Larkin, Colin (1995). - The Guinness Encyclopedia of Popular Music. - New York: Stockton Press - P.4462. - ISBN 978-0-85112-662-3.—Stanton, Scott (2003). - The Tombstone Tourist: Musicians. - New York: Pocket Books. - p.395. - ISBN 978-0-7434-6330-0.—Hicks, Jack. - "Singer Ke